In [1]:
#importe de paquetes de trabajo
import pandas as pd
import sqlite3
import pyodbc
import os
from sqlalchemy import create_engine

In [2]:
#Carga del dataset inicial
database= pd.read_excel('C:\\Users\\USUARIO\\Documents\\2.Familia\\5. Maria\\Prueba R5\\dataset.xlsx')

In [3]:
database

,num_documento_cliente,tipo_documento_cliente,codigo_tienda,total_compra,tipo_tienda,latitud_tienda,longitud_tienda,id_barrio,nombre_barrio,fecha_compra
0,6861792605025121524,1,14812028,22218042.0,Tienda especializada,3.402118,-76.525681,737998244,Canaverales Los Samanes,2022-06-15 15:49:11
1,1552970090928921643,1,14388938,21583407.0,Tienda especializada,3.468765,-76.484936,737998896,Jorge Eliecer Gaitan,2022-10-02 22:14:54
2,-4075904842556875666,1,12370698,6185349.0,Tienda especializada,3.400743,-76.524839,737998244,Canaverales Los Samanes,2022-01-10 00:27:57
3,3539339627033221084,1,10313583,10313583.0,Tienda especializada,3.369544,-76.523664,738000400,Lili,2022-09-02 16:30:24
4,5595554501572542610,1,14358071,28716142.0,Tienda especializada,3.419046,-76.497730,737996734,El Poblado,2022-03-20 02:56:49
...,...,...,...,...,...,...,...,...,...,...
94605,-6859292209449934659,2,1116,558.0,Supermercado,3.477404,-76.526705,737998252,Chipichape,2022-04-30 14:38:39
94606,-1762474661445594290,2,745,372.5,Tienda Outlet,3.414325,-76.547986,737998095,Unidad Residencial Santiago de Cali,2022-01-07 23:41:02
94607,-1762474661445594290,2,745,372.5,Supermercado,3.451204,-76.532942,737998832,San Pedro,2022-11-09 13:46:59
94608,-2708241712915270749,2,4090,2045.0,Supermercado,3.427551,-76.499991,737995769,Julio Rincón,2022-06-15 02:55:38


In [4]:
#Exploración de las fechas de compra para la creación de la dimensión de tiempo
database['fecha_compra'].min()

Timestamp('2022-01-01 00:00:31')

In [5]:
#Exploración de las fechas de compra para la creación de la dimensión de tiempo
database['fecha_compra'].max()

Timestamp('2022-12-31 23:55:58')

In [17]:
#Carga dimensión de tiempo
dim_tiempo= pd.read_excel('C:\\Users\\USUARIO\\Documents\\2.Familia\\5. Maria\\Prueba R5\\Construccion_dim_tiempo.xlsx')

In [18]:
dim_tiempo

,Fecha,Año,Trimestre,Mes,Nombre_mes,Semana,Dia,Dia_de_la_semana
0,2022-01-01,2022,1,1,enero,1,1,sábado
1,2022-01-02,2022,1,1,enero,2,2,domingo
2,2022-01-03,2022,1,1,enero,2,3,lunes
3,2022-01-04,2022,1,1,enero,2,4,martes
4,2022-01-05,2022,1,1,enero,2,5,miércoles
...,...,...,...,...,...,...,...,...
725,2023-12-27,2023,4,12,diciembre,52,27,miércoles
726,2023-12-28,2023,4,12,diciembre,52,28,jueves
727,2023-12-29,2023,4,12,diciembre,52,29,viernes
728,2023-12-30,2023,4,12,diciembre,52,30,sábado


In [8]:
#Creación dimensión de cliente
dim_cliente= database[['num_documento_cliente','tipo_documento_cliente']].drop_duplicates().groupby('num_documento_cliente').count()

In [9]:
dim_cliente

,tipo_documento_cliente
num_documento_cliente,
-9218698523982046880,1
-9214605575224426554,1
-9211522780831135515,1
-9210956249921547631,1
-9210147727599394517,1
...,...
9219183828174672272,1
9220077217609243452,1
9222313660278362882,1


In [10]:
#Creación dimensión de la tienda
dim_tienda= database[['codigo_tienda','tipo_tienda','latitud_tienda','longitud_tienda']].drop_duplicates()

In [11]:
dim_tienda

,codigo_tienda,tipo_tienda,latitud_tienda,longitud_tienda
0,14812028,Tienda especializada,3.402118,-76.525681
1,14388938,Tienda especializada,3.468765,-76.484936
2,12370698,Tienda especializada,3.400743,-76.524839
3,10313583,Tienda especializada,3.369544,-76.523664
4,14358071,Tienda especializada,3.419046,-76.497730
...,...,...,...,...
91048,11115771,Tienda especializada,3.456726,-76.533991
91062,15376312,Tienda especializada,3.451145,-76.536980
92343,1893324100,Tienda Regional,3.422141,-76.543802
92347,1420916093,Tienda Regional,3.454859,-76.519350


In [12]:
#Creación dimensión barrio
dim_barrio= database[['id_barrio','nombre_barrio']].drop_duplicates()

In [13]:
dim_barrio

,id_barrio,nombre_barrio
0,737998244,Canaverales Los Samanes
1,737998896,Jorge Eliecer Gaitan
3,738000400,Lili
4,737996734,El Poblado
5,738000407,Urbanización San Juaquín
...,...,...
74826,737998810,El Paraíso
80190,737998109,Unidad Residencial El Coliseo
81251,737996870,Colinas del Sur
84628,737996726,El Vergel


In [14]:
#creación tabla de hechos de las ventas
fact_sales_table=database[['num_documento_cliente','codigo_tienda','id_barrio','fecha_compra','total_compra']].drop_duplicates()

In [15]:
fact_sales_table

,num_documento_cliente,codigo_tienda,id_barrio,fecha_compra,total_compra
0,6861792605025121524,14812028,737998244,2022-06-15 15:49:11,22218042.0
1,1552970090928921643,14388938,737998896,2022-10-02 22:14:54,21583407.0
2,-4075904842556875666,12370698,737998244,2022-01-10 00:27:57,6185349.0
3,3539339627033221084,10313583,738000400,2022-09-02 16:30:24,10313583.0
4,5595554501572542610,14358071,737996734,2022-03-20 02:56:49,28716142.0
...,...,...,...,...,...
94605,-6859292209449934659,1116,737998252,2022-04-30 14:38:39,558.0
94606,-1762474661445594290,745,737998095,2022-01-07 23:41:02,372.5
94607,-1762474661445594290,745,737998832,2022-11-09 13:46:59,372.5
94608,-2708241712915270749,4090,737995769,2022-06-15 02:55:38,2045.0


In [16]:
#Descarga de las tablas del Datawarehouse para cargarlas poseriormente al motor de base de datos DB Browser SQLite
dim_tiempo.to_csv('dim_tiempo.csv',index=False)
dim_cliente.to_csv('dim_cliente.csv')
dim_tienda.to_csv('dim_tienda.csv',index=False)
dim_barrio.to_csv('dim_barrio.csv',index=False)
fact_sales_table.to_csv('fact_sales_table.csv',index=False)

In [20]:
#Consulta del directorio de trabajo
os.getcwd()

'C:\\Users\\USUARIO\\Documents\\2.Familia\\5. Maria\\Prueba R5'

In [21]:
#ubicación de la base de datos
path = 'C:\\Users\\USUARIO\\Documents\\2.Familia\\5. Maria\\Prueba R5\\BaseR5.db'

In [22]:
# creacion de la conexión con sqlite
sqliteConnection=sqlite3.connect(path)
cursor = sqliteConnection.cursor()

1.¿Cuáles son las tiendas con compras de al menos 100 clientes diferentes?

In [24]:
#Ejecución query que da respuesta a la primera pregunta
cursor.execute("SELECT codigo_tienda FROM (SELECT count(distinct(num_documento_cliente)) as Numero_clientes_Diferentes, codigo_tienda from fact_sales_table group by codigo_tienda HAVING Numero_clientes_Diferentes>100) ;")


In [25]:
#Respuesta a la primera pregunta
print(cursor.fetchall())

[(60,), (61,), (62,), (64,), (77,), (205,), (265,), (342,), (514,), (600,), (605,), (620,), (640,), (710,), (716,), (722,), (735,), (736,), (739,), (741,), (742,), (744,), (745,), (746,), (747,), (749,), (750,), (752,), (757,), (803,), (807,), (808,), (810,), (812,), (813,), (815,), (821,), (823,), (825,), (829,), (836,), (837,), (900,), (901,), (1089,), (1115,), (1116,), (1117,), (1119,), (1146,), (1147,), (1155,), (1156,), (1157,), (1187,), (1265,), (1266,), (1458,), (1459,), (1660,), (1669,), (1700,), (1701,), (1702,), (1703,), (1724,), (1837,), (1970,), (1974,), (1975,), (1976,), (2017,), (2018,), (2019,), (2037,), (2093,), (2134,), (2138,), (2182,), (2199,), (2337,), (2376,), (2475,), (2479,), (2522,), (2532,), (2536,), (2630,), (2683,), (2716,), (2750,), (2751,), (2752,), (2753,), (2756,), (2775,), (2843,), (2861,), (2862,), (2898,), (2899,), (2930,), (2992,), (3003,), (3005,), (3044,), (3245,), (3254,), (3256,), (3662,), (3663,), (3810,), (3937,), (3941,), (4000,), (4031,), (406

2. ¿Cuáles son los 5 barrios donde la mayor cantidad de clientes únicos realizan compras en tiendas tipo 'Tienda Regional'?

In [31]:
#Ejecución query que da respuesta a la segunda pregunta pregunta
cursor.execute("SELECT nombre_barrio from (SELECT count(distinct(num_documento_cliente)) as Numero_clientes_Diferentes, nombre_barrio from fact_sales_table f LEFT JOIN dim_barrio b on f.id_barrio=b.id_barrio LEFT JOIN dim_tienda t on f.codigo_tienda=t.codigo_tienda where tipo_tienda='Tienda Regional'group by nombre_barrio order by 1 DESC limit 5);")

In [32]:
#Respuesta a la segunda pregunta
print(cursor.fetchall())

[('Unicentro Cali',), ('Versalles',), ('El Sena',), ('Unidad Residencial Santiago de Cali',), ('Calima',)]


El código en formato SQL que da solución a esás mismas preguntas se encuentra en el repositorio y tiene por nombre "Solucion_preguntas_prueba"